In [ ]:

#import dependencies
import pandas as pd
import requests
import time
import json
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,inspect

In [ ]:
#Pull full dataset from Fantasy API and in json pretty format

query_url = f"https://api.sportsdata.io/v3/nfl/stats/json/FantasyPlayers?key=4339cb9aa27441699f1883848f498899"
response = requests.get(query_url).json()
print(json.dumps(response,indent=1))

In [ ]:
#Create Data Frame from API results (includes all fields)
df= pd.read_json(f"https://api.sportsdata.io/v3/nfl/stats/json/FantasyPlayers?key=4339cb9aa27441699f1883848f498899")

In [ ]:
#BY
#Create Data Frame that will be used to create a table of just positions that wil be used for site drop down
position = df["Position"].unique()
position_dropdown_df = pd.DataFrame(position).rename(columns={0:'Position'})
position_dropdown_df
# position_df =pd.DataFrame(position)

In [ ]:
#BY
#Create Data Frame that will be used to create table of all stats for each posiiton (5 total)
#RB

RB = df.drop(columns =["FantasyPlayerKey", "PlayerID","AuctionValue","AuctionValuePPR","AverageDraftPositionIDP","AverageDraftPositionRookie","AverageDraftPositionDynasty","AverageDraftPosition2QB"])
RB_df = RB.loc[RB["Position"]=="RB"]
RB_df
# # RB_df = df["Position"].u
# oc[concatenated_df["3_TECH_ORG_CORRECT"]=="1"]nique()

# tech_mental_health = concatenated_df.loc[concatenated_df["3_TECH_ORG_CORRECT"]=="1"]
# tech_mental_health

# position_df = pd.DataFrame(position).rename(columns={0:'Position'})
# position_df

# "Team","Position","AverageDraftPosition", "AverageDraftPositionPPR", "ByeWeek", "LastSeasonFantasyPoints", "ProjectedFantasyPoints"\
#                 from "ADP_Data"\

In [ ]:
#Create Highlights table: to appear on Home page; extract players from positions with lowest ADP for the week
highlights = df[["Name", "Team", "Position","AverageDraftPosition","AverageDraftPositionPPR","ByeWeek","LastSeasonFantasyPoints","ProjectedFantasyPoints"]].groupby("Position", as_index=False).apply(lambda df:df.sort_values("AverageDraftPosition", ascending=True)
.head(1)).droplevel(0).sort_values("Position", ascending=True)
highlights

In [ ]:
#BY
#Create Data Frame that will be used to create table of all stats for each posiiton (5 total)
#WR 

WR = df.drop(columns =["FantasyPlayerKey", "PlayerID","AuctionValue","AuctionValuePPR","AverageDraftPositionIDP","AverageDraftPositionRookie","AverageDraftPositionDynasty","AverageDraftPosition2QB"])
WR_df = WR.loc[WR["Position"]=="WR"]
WR_df

In [ ]:
#BY
#Create Data Frame that will be used to create table of all stats for each posiiton (5 total)
#TE

TE = df.drop(columns =["FantasyPlayerKey", "PlayerID","AuctionValue","AuctionValuePPR","AverageDraftPositionIDP","AverageDraftPositionRookie","AverageDraftPositionDynasty","AverageDraftPosition2QB"])
TE_df = TE.loc[TE["Position"]=="TE"]
TE_df

In [ ]:

#BY
#Create Data Frame that will be used to create table of all stats for each posiiton (5 total)
#QB

QB = df.drop(columns =["FantasyPlayerKey", "PlayerID","AuctionValue","AuctionValuePPR","AverageDraftPositionIDP","AverageDraftPositionRookie","AverageDraftPositionDynasty","AverageDraftPosition2QB"])
QB_df = QB.loc[QB["Position"]=="QB"]
QB_df

In [ ]:
DEF = df.drop(columns =["FantasyPlayerKey", "PlayerID","AuctionValue","AuctionValuePPR","AverageDraftPositionIDP","AverageDraftPositionRookie","AverageDraftPositionDynasty","AverageDraftPosition2QB"])
DEF_df = DEF.loc[DEF["Position"]=="DEF"]
DEF_df

In [ ]:
K = df.drop(columns =["FantasyPlayerKey", "PlayerID","AuctionValue","AuctionValuePPR","AverageDraftPositionIDP","AverageDraftPositionRookie","AverageDraftPositionDynasty","AverageDraftPosition2QB"])
K_df = DEF.loc[K["Position"]=="K"]
K_df

In [ ]:
#Load DF into Database NFL_Fantasy_Data
#Create connection to Postgres
connection_string = "postgres:postgres@localhost:5432/NFL_Fantasy_Data"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Create ADP table and load into DB
df.to_sql(name='ADP_Data', con=engine, if_exists='append', index=False)

In [ ]:
#BY
#Create Dropdown table and load into DB
position_dropdown_df.to_sql(name='Position_dropdown', con=engine, if_exists='append', index=False)

In [ ]:
#BY
#Create  table and load into DB
RB_df.to_sql(name='RB_Data', con=engine, if_exists='append', index=False)

In [ ]:
#BY
#Create  table and load into DB
WR_df.to_sql(name='WR_Data', con=engine, if_exists='append', index=False)

In [ ]:
#BY
#Create  table and load into DB
TE_df.to_sql(name='TE_Data', con=engine, if_exists='append', index=False)

In [ ]:
#BY
#Create  table and load into DB
QB_df.to_sql(name='QB_Data', con=engine, if_exists='append', index=False)

In [ ]:
#BY
#Create DEF table and load into DB
DEF_df.to_sql(name='DEF_Data', con=engine, if_exists='append', index=False)

In [ ]:
#BY
#Create K table and load into DB
K_df.to_sql(name='K_Data', con=engine, if_exists='append', index=False)

In [ ]:
#Create highlights table and load into DB
highlights.to_sql(name='Highlights_Data', con=engine, if_exists='append', index=False)

In [ ]:
#confirm data succesfully loaded in

pd.read_sql_query('select * from "ADP_Data"', con=engine)

In [ ]:
# Query to find the unique positions of each player; to be used for Landing Page Dropdown
pd.read_sql_query('Select distinct "Position"from "ADP_Data";', con=engine)

In [ ]:
#Note: We are creating views and displaying stats based on the Draft Pick method of selection instead of auction method. We used all position values instead of looking at individual position values exclusively (IDP)
# Query to find the stats per each position once drop down in Landing Page is selected. 
# Want this sorted by ADP Ranking; replicate this query for each position  

#DEF

wr_query = """
Select "Name", "Team","Position","AverageDraftPosition", "AverageDraftPositionPPR", "ByeWeek", "LastSeasonFantasyPoints", "ProjectedFantasyPoints"\
                from "ADP_Data"\
                where "Position"= 'WR' order by "AverageDraftPosition" asc;
"""

pd.read_sql_query(wr_query, con=engine)

In [ ]:
#Create connection to Postgres
connection_string = "postgres:postgres@localhost:5432/NFL_Fantasy_Data"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:

Base = automap_base()

In [ ]:
Base.prepare(engine, reflect= True)

In [ ]:
Base.classes.keys()